# Building MOM5

This documents the attempt to build MOM5 from source the manual way (as opposed to using Spack).

In [1]:
# Getting code
cd /scratch/tm70/$USER
git clone https://github.com/mom-ocean/MOM5 --recursive
cd MOM5

fatal: destination path 'MOM5' already exists and is not an empty directory.


## Build script

MOM5 uses a now fairly atypical build system that, I think, was carried over from previous versions.

In [2]:
# Let's try use the compile script with the ACCESS-OM version

cd exp/
# load modules - these come from bin/environs.nci
module load intel-compiler/2019.5.281 openmpi/4.0.2 netcdf/4.7.4
./MOM_compile.csh --type ACCESS-OM --platform nci --use_netcdf4

Including COSIMA version in build
... Makefile is ready.
make: Nothing to be done for 'all'.
A list of the files you checked out is in the file /scratch/tm70/ey7514/MOM5/exp/path_names ...
....................................................................................................................... Makefile is ready.
make: Nothing to be done for 'all'.
...................................................................................................................................................................... Makefile is ready.
make: Nothing to be done for 'all'.
..... Makefile is ready.
mpifort -Duse_netCDF -Duse_libMPI -DACCESS_OM -Duse_netCDF4 -DCOSIMA_VERSION  -fpp -Wp,-w -I/apps/netcdf/4.7.4/include -fno-alias -safe-cray-ptr -fpe0 -ftz -assume byterecl -i4 -r8 -traceback -nowarn -check noarg_temp_created -assume nobuffered_io -convert big_endian -grecord-gcc-switches -align all -g3 -O2 -xCORE-AVX2 -debug all -check none -I/scratch/tm70/ey7514/MOM5/exec/nci/lib_FMS 

: 1

Above failed with 
```
/scratch/tm70/ey7514/MOM5/src/accessom_coupler/mom_oasis3_interface.F90(91): error #7002: Error in opening the compiled module file.  Check INCLUDE paths.   [MOD_PRISM]
use mod_prism
```
Pretty sure it comes from the oasis coupler, but not sure how it's supposed to be installed. Double check by installing solo version

In [3]:

./MOM_compile.csh --type MOM_solo --platform nci --use_netcdf4

A list of the files you checked out is in the file /scratch/tm70/ey7514/MOM5/exp/path_names ...
....................................................................................................................... Makefile is ready.
mpifort -Duse_netCDF -Duse_libMPI -DUSE_OCEAN_BGC -DENABLE_ODA -DSPMD -DLAND_BND_TRACERS -Duse_netCDF4  -fpp -Wp,-w -I/apps/netcdf/4.7.4/include -fno-alias -safe-cray-ptr -fpe0 -ftz -assume byterecl -i4 -r8 -traceback -nowarn -check noarg_temp_created -assume nobuffered_io -convert big_endian -grecord-gcc-switches -align all -g3 -O2 -xCORE-AVX2 -debug all -check none  -c -I/scratch/tm70/ey7514/MOM5/src/shared/include -I/scratch/tm70/ey7514/MOM5/src/shared/mpp/include	/scratch/tm70/ey7514/MOM5/src/shared/mpp/mpp.F90
Using 8-byte addressing
Using PURE
Converting pointers to allocatable components
mpifort -Duse_netCDF -Duse_libMPI -DUSE_OCEAN_BGC -DENABLE_ODA -DSPMD -DLAND_BND_TRACERS -Duse_netCDF4  -fpp -Wp,-w -I/apps/netcdf/4.7.4/include -fno-alias -safe-c

That seemed to work, so it's because I'm missing something required in the OM model.

Let's check oasis.

In [5]:
# go back to my scratch and clone
cd /scratch/tm70/$USER
git clone https://github.com/ACCESS-NRI/oasis3-mct.git --recursive

Cloning into 'oasis3-mct'...
remote: Enumerating objects: 1035, done.        
remote: Counting objects: 100% (281/281), done.        
remote: Compressing objects: 100% (117/117), done.        
remote: Total 1035 (delta 121), reused 252 (delta 115), pack-reused 754 (from 1)        
Receiving objects: 100% (1035/1035), 5.47 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (448/448), done.


In [6]:
# find reference to missing module
cd oasis3-mct
find -name '*prism*'

./lib/psmile/src/mod_prism.F90
./doc/prismdoc.cls


Ok, clearly I the MOM5 ACCESS-OM is missing the oasis3-mct built module.

Let's try build it then?

In [7]:
# these commands come from build.sh
PLATFORM=nci
PROJDIR=$(pwd -P)
cd util/make_dir
echo "include ${PROJDIR}/util/make_dir/make.${PLATFORM}" > make.inc
OASIS_HOME=${PROJDIR} make -f TopMakefileOasis3 realclean
OASIS_HOME=${PROJDIR} make -j$(nproc) -f TopMakefileOasis3


/usr/bin/make -f TopMakefileOasis3 cleanlibs
make[1]: Entering directory '/scratch/tm70/ey7514/oasis3-mct/util/make_dir'
cleaning oasis3-mct libraries
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/build/lib/scrip/*
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/build/lib/mct/*
rm -f -r /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/build/lib/mctdir/*
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/build/lib/psmile.MPI1/*
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/lib/libscrip.a
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/lib/libmct.a
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/lib/libmpeu.a
rm -f /scratch/tm70/ey7514/oasis3-mct/compile_oa3-mct/lib/libpsmile.MPI1.a
make[1]: Leaving directory '/scratch/tm70/ey7514/oasis3-mct/util/make_dir'
rm -f /scratch/tm70/ey7514/oasis3-mct/util/make_dir/COMP.log /scratch/tm70/ey7514/oasis3-mct/util/make_dir/COMP.err
/usr/bin/make -f TopMakefileOasis3 makedirs
make[1]: Entering directory '/scratch/t

In [8]:
find ../../ -name '*prism*.mod'

../../compile_oa3-mct/build/lib/psmile.MPI1/mod_prism.mod


**ignore below**

OK, I think I've found the relevant module file, but I'm not sure how to include it into the MOM_compile.sh script...

I've found this line:
```
set lib_include_dirs = "$root/include $code_dir/shared/include $code_dir/shared/mpp/include"
```
Maybe I could soft link the module into there?

**ignore above**

Ok, I had a look at the template that `MOM_compile.csh` uses as a template:

In [11]:
pwd

/scratch/tm70/ey7514/oasis3-mct


In [12]:
cd /scratch/tm70/$USER/MOM5
grep OASIS_ROOT bin/mkmf.template.nci

ifneq ($(OASIS_ROOT),)
INCLUDE  += -I$(OASIS_ROOT)/Linux/build/lib/psmile.MPI1 \
            -I$(OASIS_ROOT)/Linux/build/lib/pio \
            -I$(OASIS_ROOT)/Linux/build/lib/mct
ifneq ($(OASIS_ROOT),)
LIBS += -L$(OASIS_ROOT)/Linux/lib -lpsmile.MPI1 -lmct -lmpeu -lscrip


So it looks like OASIS_ROOT needs to be set as an environment variable. But, it expects a different layout than what currently exists:

In [13]:
ls ../oasis3-mct/compile_oa3-mct/
ls ../oasis3-mct/compile_oa3-mct/build/lib

build  lib


mct  mctdir  psmile.MPI1  scrip


Let's fix that with a hackey softlink

In [14]:
ln -s $(realpath ../oasis3-mct/compile_oa3-mct/) ../oasis3-mct/Linux
tree -d ../oasis3-mct/Linux

../oasis3-mct/Linux
├── build
│   └── lib
│       ├── mct
│       ├── mctdir
│       │   ├── doc
│       │   │   └── texsrc
│       │   ├── examples
│       │   │   ├── climate_concur1
│       │   │   ├── climate_sequen1
│       │   │   └── simple
│       │   ├── m4
│       │   ├── mct
│       │   ├── mpeu
│       │   ├── mpi-serial
│       │   └── protex
│       ├── psmile.MPI1
│       └── scrip
└── lib

18 directories


Let's try the build again with the environment variable set

In [15]:
cd exp

export OASIS_ROOT=$(realpath ../../oasis3-mct)

OASIS_ROOT=$OASIS_ROOT csh -x ./MOM_compile.csh --type ACCESS-OM --platform nci --use_netcdf4 # csh -x to see how the variables were set

set platform = gfortran
set type = MOM_solo
set unit_testing = 0
set help = 0
set debug = 0
set repro = 0
set use_netcdf4 = 0
set environ = 1
set cosima_version = unset
set argv = ( `getopt -u -o h -l type: -l platform: -l help -l unit_testing -l debug -l repro -l use_netcdf4 -l no_environ -l no_version --  $*` )
getopt -u -o h -l type: -l platform: -l help -l unit_testing -l debug -l repro -l use_netcdf4 -l no_environ -l no_version -- --type ACCESS-OM --platform nci --use_netcdf4
if ( 0 != 0 ) then
while ( --type != -- )
switch ( --type )
set type = ACCESS-OM
shift argv
breaksw
shift argv
end
while ( --platform != -- )
switch ( --platform )
set platform = nci
shift argv
breaksw
shift argv
end
while ( --use_netcdf4 != -- )
switch ( --use_netcdf4 )
set use_netcdf4 = 1
breaksw
shift argv
end
while ( -- != -- )
shift argv
help:
if ( 0 ) then
set root = /scratch/tm70/ey7514/MOM5
set code_dir = /scratch/tm70/ey7514/MOM5/src
set executable = /scratch/tm70/ey7514/MOM5/exec/nci/ACCESS-OM/fms_A

: 1

Ok past the old error, but a new one has shown up (another missing module):
```
/scratch/tm70/ey7514/MOM5/src/accessom_coupler/ocean_solo.F90(111): error #7002: Error in opening the compiled module file.  Check INCLUDE paths.   [ACCESSOM2_MOD]
  use accessom2_mod, only : accessom2_type => accessom2
```
And looking again at the makefile template, it's *probably* because because LIBACCESSOM2_ROOT is unset.

In [17]:
grep LIBACCESSOM2_ROOT ../bin/mkmf.template.nci

ifneq ($(LIBACCESSOM2_ROOT),)
INCLUDE  += -I$(LIBACCESSOM2_ROOT)/oasis3-mct/Linux/build/lib/psmile.MPI1 \
            -I$(LIBACCESSOM2_ROOT)/oasis3-mct/Linux/build/lib/mct \
            -I$(LIBACCESSOM2_ROOT)/build/include
ifneq ($(LIBACCESSOM2_ROOT),)
LIBS += -L$(LIBACCESSOM2_ROOT)/build/lib -laccessom2


So if I was smarter, I would've noticed that oasis3-mct is also included in `libaccessom2`, so I could've started from there instead of building `oasis3-mct` seperately. Let's do that.

In [4]:
cd /scratch/tm70/$USER
git clone https://github.com/COSIMA/libaccessom2.git --recursive
cd libaccessom2/

# there's a build.sh and build_on_gadi.sh in there, so lets try run that
bash build_on_gadi.sh

fatal: destination path 'libaccessom2' already exists and is not an empty directory.
mpifort executable found: 
Will assume system MPI implementation is sound. Remove mpifort from PATH to automatically configure MPI
-- Configuring done (0.5s)
-- Generating done (0.9s)
-- Build files have been written to: /scratch/tm70/ey7514/libaccessom2/build
/half-root/usr/bin/cmake -S/scratch/tm70/ey7514/libaccessom2 -B/scratch/tm70/ey7514/libaccessom2/build --check-build-system CMakeFiles/Makefile.cmake 0
/half-root/usr/bin/cmake -E cmake_progress_start /scratch/tm70/ey7514/libaccessom2/build/CMakeFiles /scratch/tm70/ey7514/libaccessom2/build//CMakeFiles/progress.marks
make  -f CMakeFiles/Makefile2 all
make[1]: Entering directory '/scratch/tm70/ey7514/libaccessom2/build'
make  -f CMakeFiles/datetime-fortran.dir/build.make CMakeFiles/datetime-fortran.dir/depend
make[2]: Entering directory '/scratch/tm70/ey7514/libaccessom2/build'
cd /scratch/tm70/ey7514/libaccessom2/build && /half-root/usr/bin/cmake

In [5]:
ls oasis3-mct

ChangeLog  Linux  Makefile  README.md  doc  examples  lib  svn.info  util


looks like the cmake build produces the expected directory structure in oasis3-mct, so no hackey soft links needed.

Let's restart our build again

In [6]:
cd /scratch/tm70/ey7514/MOM5/exp
LIBACCESSOM2_ROOT=/scratch/tm70/$USER/libaccessom2 csh ./MOM_compile.csh --type ACCESS-OM --platform nci --use_netcdf4

Including COSIMA version in build
... Makefile is ready.
make: Nothing to be done for 'all'.
A list of the files you checked out is in the file /scratch/tm70/ey7514/MOM5/exp/path_names ...
....................................................................................................................... Makefile is ready.
mpifort -Duse_netCDF -Duse_libMPI -DACCESS_OM -Duse_netCDF4 -DCOSIMA_VERSION  -fpp -Wp,-w -I/apps/netcdf/4.7.4/include -I/scratch/tm70/ey7514/libaccessom2/oasis3-mct/Linux/build/lib/psmile.MPI1 -I/scratch/tm70/ey7514/libaccessom2/oasis3-mct/Linux/build/lib/mct -I/scratch/tm70/ey7514/libaccessom2/build/include -fno-alias -safe-cray-ptr -fpe0 -ftz -assume byterecl -i4 -r8 -traceback -nowarn -check noarg_temp_created -assume nobuffered_io -convert big_endian -grecord-gcc-switches -align all -g3 -O2 -xCORE-AVX2 -debug all -check none  -c -I/scratch/tm70/ey7514/MOM5/src/shared/include -I/scratch/tm70/ey7514/MOM5/src/shared/mpp/include	/scratch/tm70/ey7514/MOM5/src/shar

And there we have it! Our we have our compiled binary in:

In [7]:
find /scratch/tm70/$USER -name 'fms_ACCESS-OM.x'

/scratch/tm70/ey7514/MOM5/exec/nci/ACCESS-OM/fms_ACCESS-OM.x
